In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import click
import logging
import statsmodels.formula.api as sm
import patsy

In [ ]:
interim = Path('../../data/interim')
events = pd.read_pickle(Path(interim) / 'events.pkl')

In [ ]:
events.HOME_TEAM_ID

In [ ]:
print(events.dtypes.to_string())

In [ ]:
base = events.groupby([
    'GAME_ID', 'HOME_TEAM_ID', 'AWAY_TEAM_ID', 'BAT_HAND_CD', 'year'
])[['H', 'EVENT_OUTS_CT']].sum()

In [ ]:
base_with_park = pd.merge(base.reset_index(), game_logs[['GAME_ID', 'ParkID']], on=['GAME_ID'], how='left')

In [ ]:
base_with_park

In [ ]:
Home = base_with_park.groupby(['HOME_TEAM_ID', 'year', 'ParkID', 'BAT_HAND_CD']).agg({
    'H': 'sum',
    'EVENT_OUTS_CT': 'sum',
    'GAME_ID': 'nunique', 
})

Home = Home.reset_index()
Home.rename(columns={'EVENT_OUTS_CT': 'outs', 'HOME_TEAM_ID': 'TEAM_ID', 'GAME_ID': 'home_G'}, inplace=True)
Home = Home.set_index(['TEAM_ID', 'year', 'BAT_HAND_CD'])

Home['home_HPO'] = Home['H'] / Home['outs']

In [ ]:
Road = base_with_park.groupby(['AWAY_TEAM_ID', 'year', 'BAT_HAND_CD']).agg({
    'H': 'sum',
    'EVENT_OUTS_CT': 'sum',
})

Road = Road.reset_index()
Road.rename(columns={'EVENT_OUTS_CT': 'outs', 'AWAY_TEAM_ID': 'TEAM_ID'}, inplace=True)
Road = Road.set_index(['TEAM_ID', 'year', 'BAT_HAND_CD'])

Road['road_HPO'] = Road['H'] / Road['outs']

In [ ]:
compare = pd.merge(Home[['ParkID', 'home_G', 'home_HPO']], Road[['road_HPO']], on=['TEAM_ID', 'year', 'BAT_HAND_CD'])

In [ ]:
compare['factor'] = compare['home_HPO'] / compare['road_HPO']

In [ ]:
compare.groupby(['ParkID', 'year', 'BAT_HAND_CD']).size()

In [ ]:
compare[compare['home_G'] > 28]

In [ ]:
compare[compare['home_G'] > 30].sort_values('factor').tail(50)

In [ ]:
park_factors['factor_h_year'] = park_factors['home_h_factor'] / park_factors['road_h_factor']
park_factors['factor_pa_year'] = park_factors['home_pa_factor'] / park_factors['road_pa_factor']
park_factors['factor_hppa_year'] = park_factors['factor_h_year'] / park_factors['factor_pa_year']
park_factors = park_factors.reset_index().set_index(['ParkID', 'year'])
park_factors = park_factors.sort_values(['ParkID', 'year'])

park_factors['h_factor'] = park_factors.groupby('ParkID')[['factor_h_year']].transform(lambda x: x.rolling(10, 1).mean())
park_factors['pa_factor'] = park_factors.groupby('ParkID')[['factor_pa_year']].transform(lambda x: x.rolling(10, 1).mean())
park_factors['hppa_factor'] = park_factors.groupby('ParkID')[['factor_hppa_year']].transform(lambda x: x.rolling(10, 1).mean())

park_factors['h_factor'] = park_factors['h_factor'].clip(.8, 1.25)

idx = park_factors.groupby(['ParkID', 'year'])['home_g'].transform(max) == park_factors['home_g']
park_factors = park_factors.loc[idx]
park_factors = park_factors.loc[park_factors.home_g >= 20]

return park_factors